In [ ]:
from BiP_GA import ga_main
import numpy as np
import os
import matplotlib.pyplot as plt

## Example of permutation coding.
### Solve traveling salesman problem

In [ ]:
def plot_root(method,gen,count):
    plt.scatter(0,0,c="g")
    plt.scatter(position[:,0],position[:,1],c="b")
    plt.plot([0,position[method.best_ind[0],0]],[0,position[method.best_ind[0],1]],c = "r")
    plt.plot([0,position[method.best_ind[-1],0]],[0,position[method.best_ind[-1],1]],c = "r")
    plt.plot(position[method.best_ind,0],position[method.best_ind,1],c ="r")
    plt.title("Generation :{}".format(gen))
    plt.savefig("./tsp_graph/root_plot_{0:04d}.png".format(count),format="png")
    plt.close()

In [ ]:
"""initialize"""
n_parents = 15        # the number of parents
l_gen = 50            # the length of gen
n_pop   = 100         # the number of popuration
e_size = 5            # Elete size for elete selection
t_size =10            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

perm = ga_main.permutation_ga(l_gen = l_gen, n_pop= n_pop, n_parents=n_parents,  # Initialize
             pb_mut=pb_mut, pb_crs = pb_crs) 

"""Ratios of each methods in operations"""
crs_ratio = [0,0,1,0]  # [cycle, op_order, order_based, position_based]
mut_ratio = [0.25,0.25,0.25,0.25] # [swap, inversion, scramble, translocation]

"""Set parameters"""
perm.set_params(crs_ratio = crs_ratio, mut_ratio = mut_ratio)

perm.show_params()  # print all parameters

max_generation = 1500 # Max generation

In [ ]:
"""Initial position"""
position = np.random.rand(l_gen,2)*100 

stop_limit = 100
count = 0
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        perm.make_init_generation() # make 1st generation population
        fitness = perm.calc_dist_fitness(position)
        perm.set_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(perm.fitness)))
        perm.get_best_individuals()                        # get best individual in this generation
        count = count +1
        plot_root(perm,gen,count)
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = perm.tournament_selection(t_size=t_size, population = perm.inds)
    ### Elete selection
    kb = perm.elete_selection(p_size=e_size, population = perm.inds)   
    
     
    offspring = np.empty([0,perm.l_gen]) ### next generation population
    while offspring.shape[0] != perm.n_pop - e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        perm.Crossover(parents[p1], parents[p2])
        """Mutation"""
        perm.Mutation(parent=perm.child)
        #perm.GMutation(parent=perm.child, generation=gen, extra_mut=0.5, cycle=100)
        offspring = np.append(offspring, perm.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0).astype(int)
    perm.set_individuals(offspring)
    
    fitness = perm.calc_dist_fitness(position)
    perm.set_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(perm.fitness)))
    perm.get_best_individuals()
    
    """Plot graph"""
    if perm.best_fit_list[gen-2] < perm.best_fit_list[gen-1]:
        count = count +1
        plot_root(perm,gen,count)
    
    """Check converge"""
    if perm.best_fit_list[gen-1] == perm.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif perm.best_fit_list[gen-1] != perm.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        
    

In [ ]:
%matplotlib inline

plt.scatter(0,0,c="g")
plt.scatter(position[:,0],position[:,1],c="b")
plt.plot([0,position[perm.best_ind[0],0]],[0,position[perm.best_ind[0],1]],c = "r")
plt.plot([0,position[perm.best_ind[-1],0]],[0,position[perm.best_ind[-1],1]],c = "r")
plt.plot(position[perm.best_ind,0],position[perm.best_ind,1],c ="r")


In [ ]:
"""Make gif"""
import os
os.system("convert -layers optimize -loop 0 -delay 40 ./tsp_graph/root_plot_*.png ./tsp_graph/root_plot.gif")

## Example of binary coding.
### Solve OneMax problem

In [ ]:
"""initialize"""
n_parents = 15        # the number of parents
l_gen = 200            # the length of gen
n_pop   = 100         # the number of popuration
e_size = 5            # Elete size for elete selection
t_size =10            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

bina = ga_main.binary_ga(l_gen = l_gen, n_pop= n_pop, n_parents=n_parents,  
            pb_mut=pb_mut, pb_crs = pb_crs) 

"""Ratios of each methods in operations"""
crs_ratio = [0,1,0]  # [op, tp, uniform]
mut_ratio = [0.25,0.25,0.25,0.25] # [substitution, inversion, scramble, translocation]

"""Set parameters"""
bina.set_params(crs_ratio = crs_ratio, mut_ratio = mut_ratio)

bina.show_params()  # print all parameters

max_generation = 1500 # Max generation

In [ ]:
"""Initial position"""
stop_limit = 100
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        bina.make_init_generation(n_1 = "random") # make 1st generation population
        fitness = bina.calc_onemax_fitness()
        bina.set_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(bina.fitness)))
        bina.get_best_individuals()                        # get best individual in this generation
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = bina.tournament_selection(t_size=t_size, population = bina.inds)
    ### Elete selection
    kb = bina.elete_selection(p_size=e_size, population = bina.inds)   
    
     
    offspring = np.empty([0,bina.l_gen]) ### next generation population
    while offspring.shape[0] != bina.n_pop - e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        bina.Crossover(parents[p1], parents[p2])
        """Mutation"""
        bina.Mutation(parent=bina.child)
        #bina.GMutation(parent=bina.child, generation=gen, extra_mut=0.5, cycle=100)
        offspring = np.append(offspring, bina.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0).astype(int)
    bina.set_individuals(offspring)
    
    fitness = bina.calc_onemax_fitness()
    bina.set_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(bina.fitness)))
    bina.get_best_individuals()
    
    """Check converge"""
    if bina.best_fit_list[gen-1] == bina.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif bina.best_fit_list[gen-1] != bina.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        
    

## Example of binary + permutation coding.
### Solve one sort problem

In [ ]:
"""initialize"""
n_parents = 10        # the number of parents
l_gen = 27            # the length of gen
n_pop   = 20          # the number of popuration
e_size = 1            # Elete size for elete selection
t_size = 3            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

n_1 = 10 # the number of 1 

bip = ga_main.bip_ga(l_gen = l_gen, n_pop = n_pop, n_parents=n_parents,  
            pb_mut=pb_mut, pb_crs = pb_crs) 

"""Ratios of each methods in operations"""
#crs_ratio = [1]  # When b+p type crs_ratio is needless to set
mut_ratio = [0.25,0.25,0.25,0.25] # [bp_swap, inversion, scramble, translocation]

"""Set parameters"""
bip.set_params(mut_ratio = mut_ratio)

bip.show_params()  # print all parameters

max_generation = 1500 # Max generation

In [ ]:
"""Initial position"""
stop_limit = 100
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        bip.make_init_generation(n_1 = n_1) # make 1st generation population
        fitness = bip.calc_sort_fitness()
        bip.set_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(bip.fitness)))
        bip.get_best_individuals()                        # get best individual in this generation
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = bip.tournament_selection(t_size=t_size, population = bip.inds)
    ### Elete selection
    kb = bip.elete_selection(p_size=e_size, population = bip.inds)   
    
     
    offspring = np.empty([0,bip.l_gen]) ### next generation population
    while offspring.shape[0] != bip.n_pop - e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        bip.Crossover(parents[p1], parents[p2])
        """Mutation"""
        bip.Mutation(parent=bip.child)
        #bip.GMutation(parent=bip.child, generation=gen, extra_mut=0.5, cycle=100)
        offspring = np.append(offspring, bip.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0).astype(int)
    bip.set_individuals(offspring)
    
    fitness = bip.calc_sort_fitness()
    bip.set_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(bip.fitness)))

    bip.get_best_individuals()
    
    """Check converge"""
    if bip.best_fit_list[gen-1] == bip.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif bip.best_fit_list[gen-1] != bip.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        
    

## Example of multi binary + permutation coding.
### Solve one sort problem

In [ ]:
"""initialize"""
n_parents = 10        # the number of parents
l_gen = 27            # the length of gen
n_pop   = 20          # the number of popuration
e_size = 1            # Elete size for elete selection
t_size = 3            # tournament size for tournament selection

pb_crs = 1            # probability of crossover
pb_mut = 0.1          # probability of mutation

mbip = ga_main.multi_bip_ga(l_gen = l_gen, n_pop = n_pop, n_parents=n_parents,  
            pb_mut=pb_mut, pb_crs = pb_crs) 

"""Ratios of each methods in operations"""
#crs_ratio = [1]  # When b+p type crs_ratio is needless to set
mut_ratio = [0.25,0.25,0.25,0.25] # [bp_swap, inversion, scramble, translocation]

"""Set parameters"""
mbip.set_params(mut_ratio = mut_ratio)

mbip.show_params()  # print all parameters

max_generation = 1500 # Max generation

n = [9,6,5,3,4] # gen has {0:9, 1:6, 2:5, 3:3, 4:4} best fit is 820

In [ ]:
"""Initial position"""
stop_limit = 100
stop_count = 0

for gen in range(1,max_generation):
    print(" ----------  Generation : {} ---------".format(gen))
    if gen == 1:
        mbip.make_init_generation(n_list= n) # make 1st generation population
        fitness = mbip.calc_sort_fitness()
        mbip.set_fitness(fitness)                          # calculate fitness
        print("best fitness : "+ str(max(mbip.fitness)))
        mbip.get_best_individuals()                        # get best individual in this generation
        continue
    
    
    """Selection"""
    ### Tournament selection
    parents = mbip.tournament_selection(t_size=t_size, population = mbip.inds)
    ### Elete selection
    kb = mbip.elete_selection(p_size=e_size, population = mbip.inds)   
    
     
    offspring = np.empty([0,mbip.l_gen]) ### next generation population
    while offspring.shape[0] != mbip.n_pop - e_size:
        p1, p2 = np.random.choice(parents.shape[0],2,replace=False) 
        """Crossover"""
        mbip.Crossover(parents[p1], parents[p2])
        """Mutation"""
        #mbip.Mutation(parent=mbip.child)
        mbip.GMutation(parent=mbip.child, generation=gen, extra_mut=0.5, cycle=20)
        offspring = np.append(offspring, mbip.child.reshape(1,-1),axis=0)
    
    
    offspring = np.append(offspring,kb,axis=0).astype(int)
    mbip.set_individuals(offspring)
    
    fitness = mbip.calc_sort_fitness()
    mbip.set_fitness(fitness)                          # calculate fitness
    print("best fitness : "+ str(max(mbip.fitness)))

    mbip.get_best_individuals()
    
    """Check converge"""
    if mbip.best_fit_list[gen-1] == mbip.best_fit_list[gen-2]:
        stop_count = stop_count +1
    elif mbip.best_fit_list[gen-1] != mbip.best_fit_list[gen-2]:
        stop_count = 0
    
    if stop_count == stop_limit:
        print("GA is converged. Finish evolutional roop.")
        break
        

In [ ]:
"""Show result"""
print(mbip.best_ind) # best individual of the newest generation
print(mbip.best_ind_list) # best individuals of each generation
print(mbip.best_fit) # best fitness of the newest generation
print(mbip.best_ind_list) # best fitness of each generation